In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import math
import pandas as pd

In [17]:
# ethurl = "https://api.etherscan.io/api?module=account&action=balance&address=0xE9df388B01369d972c53ABeb446C5Ae646500222&tag=latest&apikey=FTEB6TINTPNKPMSBJ4IGN9EYR1CAHD2D8A"

# eth_response = requests.get(ethurl, headers=my_headers)
# print(eth_response.json())

{'status': '1', 'message': 'OK', 'result': '2749593091425000'}


In [2]:

def twitter_api(q_string):
    access_token = "AAAAAAAAAAAAAAAAAAAAAJesOgEAAAAAhjjjXkVka1TtQ5lxdzIEJtKk6Tw%3DlkuKd4yJE1x29rdaMdvvvh6BoCxFOL8knOxSWL42QG2nU2auvL"
    my_headers = {'Authorization' : f'Bearer {access_token}'}

    tweet_fields = "&tweet.fields=text&expansions=author_id"
    user_fields = "&user.fields=description,id,location,name,profile_image_url,protected,public_metrics,url,username,verified,withheld"

    response = requests.get(f'https://api.twitter.com/2/tweets/search/recent?query={q_string}{tweet_fields}{user_fields}', headers=my_headers)
    return response.json()


In [6]:
df = pd.read_csv('reddit_scrape/user_addresses_removed_dup.csv')
address_list = df['address'].tolist()
users_list = df['user name'].tolist()
addr_len = len(address_list)

In [7]:
def gen_ugly(i,addr):

    ugly_res = {
        "includes": {
            "users": [
                {
                    "name": f"Kevin J{i}",
                    "url": "",
                    "protected": False,
                    "id": f"484269760_{j}",
                    "description": f"{addr}_{j}",
                    "location": f"Naija{j}",
                    "username": f"jeemmai{j}",
                    "public_metrics": {
                        "followers_count": 40,
                        "following_count": 294,
                        "tweet_count": 619,
                        "listed_count": 1
                    },
                    "profile_image_url": f"{j}https://pbs.twimg.com/profile_images/1319230711245754369/MsxQbjnu_normal.jpg",
                    "verified": False
                }
            ]
        },
        "meta": {
            "newest_id": "1472231060364791812",
            "oldest_id": "1471780673107148802",
            "result_count": 4
        }
    }
    return ugly_res

In [8]:
def makeCalls(ind,df):
    #tweet_json = twitter_api(address_list[i])
    tweet_json = gen_ugly(ind-1,address_list[ind-1])
    print(f"Running call for {ind - 1}")
    if(tweet_json.get("meta").get("result_count") == 0):
        tweet_person = {"description":"", "location":"","name":"","username":"","profile_image_url":""}
    else:
        
        tweet_person = tweet_json.get("includes").get("users")[0]

    tweet_desc.append(tweet_person.get("description"))
    tweet_location.append(tweet_person.get("location"))
    tweet_realname.append(tweet_person.get("name"))
    tweet_username.append(tweet_person.get("username"))
    tweet_avatar.append(tweet_person.get("profile_image_url"))
    #data={"user name": users_list[ind], "address": address_list[ind], "description": tweet_desc, "location": tweet_location,"twitter_name": tweet_realname, "twitter_username": tweet_username,"avatar": tweet_avatar}
    # df = df.append(data, ignore_index=True)
    # return df  
    # df = pd.DataFrame(data={"user name": users_list[450:601], "address": address_list[450:601], "description": tweet_desc, "location": tweet_location,"twitter_name": tweet_realname, "twitter_username": tweet_username,"avatar": tweet_avatar})
    # df.to_csv("twitter_data.csv", index=False)


In [9]:
batch_index = 0
max_calls = 450
sleep_time = 15
df = pd.DataFrame()

global_tweet_desc = []
global_tweet_location = []
global_tweet_realname = []
global_tweet_username = []
global_tweet_avatar = []


for i in range(0,math.ceil((addr_len - 1) / max_calls)):
    tweet_desc = []
    tweet_location = []
    tweet_realname = []
    tweet_username = []
    tweet_avatar = []
    print(f"Second loop {i * max_calls} => {addr_len}")
    for j in range((i * max_calls) + 1,addr_len + 1): # if first +1 is removed, loop will end on 16200 instead of 16374
        makeCalls(j,df)
        print(j)       

        if(j % max_calls == 0):
            print("Getting to sleep")
            time.sleep(3)
            break
    
    global_tweet_desc += tweet_desc
    global_tweet_location += tweet_location
    global_tweet_realname += tweet_realname
    global_tweet_username += tweet_username
    global_tweet_avatar += tweet_avatar


print("----")
print(len(address_list))
print("BBBBBBB")
print(len(global_tweet_desc))
df = pd.DataFrame(data={"user name": users_list, "address": address_list, "description": global_tweet_desc, "location": global_tweet_location,"twitter_name": global_tweet_realname, "twitter_username": global_tweet_username,"avatar": global_tweet_avatar})
df.to_csv("twitter_data.csv", index=False)



Second loop 0 => 8557
Running call for 0
1
Running call for 1
2
Running call for 2
3
Running call for 3
4
Running call for 4
5
Running call for 5
6
Running call for 6
7
Running call for 7
8
Running call for 8
9
Running call for 9
10
Running call for 10
11
Running call for 11
12
Running call for 12
13
Running call for 13
14
Running call for 14
15
Running call for 15
16
Running call for 16
17
Running call for 17
18
Running call for 18
19
Running call for 19
20
Running call for 20
21
Running call for 21
22
Running call for 22
23
Running call for 23
24
Running call for 24
25
Running call for 25
26
Running call for 26
27
Running call for 27
28
Running call for 28
29
Running call for 29
30
Running call for 30
31
Running call for 31
32
Running call for 32
33
Running call for 33
34
Running call for 34
35
Running call for 35
36
Running call for 36
37
Running call for 37
38
Running call for 38
39
Running call for 39
40
Running call for 40
41
Running call for 41
42
Running call for 42
43
Running 